In [1]:
import sys, importlib
sys.path.append('../scripts')
import pandas as pd

In [2]:
from prototypical_fit import fit
importlib.reload(sys.modules['prototypical_fit'])


<module 'prototypical_fit' from 'E:\\GradThesisTmpo\\Plitical-changes-on-R-canada--Reddit-\\notebooks\\../scripts\\prototypical_fit.py'>

In [3]:
# Read data file (2019&2020)
data = pd.read_feather('../data/feather_files/data2019clean.feather')
data = pd.concat((data,\
        pd.read_feather('../data/feather_files/data2020clean.feather')))
# Take test data out
test = data.query("tclass == 'test'").reset_index(drop=True).rename(columns={'text':'X'})[['X']]
data = data.query("tclass == 'political' or tclass=='nonpolitical'").reset_index(drop=True)
data['X'] = data.text
data['Y'] = pd.get_dummies(data.tclass).drop('nonpolitical', axis=1).values.ravel()
data = data[['X','Y']]
print("A data sample")
print(data.head(1))
print(f"The shape of the data is {data.shape}")
print('Proportions of positive (political) and negative labels:')
print(data.Y.value_counts() / data.shape[0]) 

A data sample
                                                                      X  Y
0      crime   democrat slogan    presidential election  anything        1
The shape of the data is (505676, 2)
Proportions of positive (political) and negative labels:
1    0.683711
0    0.316289
Name: Y, dtype: float64


In [10]:
%%time
fit(data, 'Reddit', k=900, log_to_file=True)

###################awe########################################
Starting fit
A data sample
+--------+------------------------------------------------------------------------------------------------------------------------------+-----+
|        | X                                                                                                                            |   Y |
|--------+------------------------------------------------------------------------------------------------------------------------------+-----|
|  92940 | note show  comey write  memo  month  trump fire                                                                              |   1 |
|  44073 | would   difficult   select     mate                                                                                          |   1 |
| 227303 | rumor spread   banquet  czar nicholas  coronation   would   enough beer  pretzel  everyone    result crowd crush kill almost |   0 |
+--------+------------------------------------

Reddit


Taking 20.0% test subset.
The resulting train shape is (404540, 2) and test shape is (101136, 2)
dividing data into classes
Joining the series of text into one string per category
Dividing those long strings into lists of words
Word Cloud
class 1
Word Cloud
class 0
Counting the occurrences of each word per class
Total umber of words (training+validation) is:
Class 1: 2527675, Class 0: 3903092
Number of distinct training words for each class is
25052 and 48967
Visualizing the top 15 common words in each category [latex code below]
+--------------------------------------+
| Most common 15 words in each category  |
+----------------------+---------------+
|       class 1        |    class 0    |
+----------------------+---------------+
|        trump         |      like     |
|        biden         |      make     |
|        house         |      know     |
|     coronavirus      |     think     |
|         call         |      tell     |
|       democrat       |     would     |
|      pres

Pandas Apply:   0%|          | 0/25052 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/48967 [00:00<?, ?it/s]

...

Counting the occurrence of the chosen words inside each of the posts.
The resulting dataframe is of shape (number of posts)x(2k).


Pandas Apply:   0%|          | 0/404540 [00:00<?, ?it/s]

saved as wp_in_u_900 with the dimension of (404540, 1)
Transforming the previous variable into a dataframe. saved as wp_proto_900


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

OSError: [WinError 1450] Insufficient system resources exist to complete the requested service

hello
hello
hello
hello
Wall time: 392 µs
